In [2]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
import Functions as f
from sklearn.model_selection import train_test_split

1. Start by splitting the data 4:1 as training and validation randomly (for grading purposes please use np.random.seed(1)).

In [3]:
data = pd.read_csv('data/cars-mpg.csv').to_numpy()
np.random.seed(1)
np.random.shuffle(data)

x_train, x_val, y_train, y_val = train_test_split(data[:,1:], data[:,0], test_size=0.20, random_state=None, shuffle=False)

2. Implement the forward selection algorithm as discussed in Lecture 6 (see lecture notes for
details). In the loop use the training MSE to find the best model in each iteration. The
algorithm should produce p + 1 models M0, ..., Mp, where Mi is the best model using
i features. In terms of output, an alternative could be to let the algorithm produce a
p-dimensional vector where its first entry is the feature in M1, its second entry is the new
feature in M2 etc.

In [4]:
num_of_features = x_train.shape[1]
# the algorithm can be found inside the Functions.py file
models = f.feature_selection(x_train, y_train, num_of_features, LinearRegression)
for model in models:
    numpy_model = np.array(model[0])
    print("Model features (index from the training data): ", model[1])

Model features (index from the training data):  []
Model features (index from the training data):  [3]
Model features (index from the training data):  [3, 5]
Model features (index from the training data):  [3, 5, 6]
Model features (index from the training data):  [3, 5, 6, 4]
Model features (index from the training data):  [3, 5, 6, 4, 1]
Model features (index from the training data):  [3, 5, 6, 4, 1, 2]
Model features (index from the training data):  [3, 5, 6, 4, 1, 2, 0]


In [6]:
mse_values = []
features_in_order = []
val_models = []
x_val_base = np.ones((x_val.shape[0], 1))
for i in range(len(models) - 1):
    # I am skipping the first model since it's a model with no features
    model = models[i + 1]
    numpy_model = np.array(model[0])
    feature_indexes = model[1]
    feature_index = feature_indexes[-1]
    x_val_base_with_new_col = np.hstack((x_val_base, x_val[:, feature_index].reshape((-1, 1))))
    x_val_base = x_val_base_with_new_col
    X_val_subset = x_val[:, feature_indexes]
    linreg = LinearRegression()
    linreg.fit(numpy_model, y_train)
    y_pred_val = linreg.predict(x_val_base)
    val_mse = ((y_pred_val - y_val) ** 2).mean()
    mse_values.append(val_mse)
    features_in_order.append(feature_index)
    val_models.append(x_val_base)

print(mse_values)
print("The smallest MSE value calculated from the validation set is: ", min(mse_values))
print("which corresponds to the model: Model", mse_values.index(min(mse_values)) + 1, "with features", features_in_order[:mse_values.index(min(mse_values)) + 1])
print("The features in order of importance are: ", features_in_order, "where the first feature is the most important one.")

[16.661741658298318, 11.22429612906207, 10.619713011450296, 10.957677532220014, 10.887335885968636, 10.853301569940479, 10.731411881246974]
The smallest MSE value calculated from the validation set is:  10.619713011450296
which corresponds to the model: Model 3 with features [3, 5, 6]
The features in order of importance are:  [3, 5, 6, 4, 1, 2, 0] where the first feature is the most important one.
